In [ ]:
# SECTION 2
# The only issue that are lacking some atm variables

In [1]:
import cfgrib
from ecmwf.opendata import Client
import ipywidgets as widgets
from IPython.display import display
from Magics import macro as magics
from Magics.macro import *
import numpy as np
import pandas as pd
import requests
import rioxarray as rxr
import xarray as xr

# PARAMETERS TO BE IMPORTED FROM MAIN WIDGET

In [34]:
dates_to_download = [ '20230823', '20230824']

initial_lat_lon = (38, -90)
final_lat_lon = (60, 90)

In [17]:
str(dates_to_download[0])

'20230818'

# VARIABLE INITIALIZATION

In [11]:
# sea surface variables: mslp; sst
ss_variables = ['msl', '2t']

min_lat = initial_lat_lon[0]
min_lon = initial_lat_lon[1]

max_lat = final_lat_lon[0]
max_lon = final_lat_lon[1]

# DOWNLOAD DATA

In [36]:
print('Downloading data...')
# high res. variables
for var in ['msl', '2t', 'tp']:
    print(var)
    for t in  dates_to_download:
        #print(var + ': ' + t)
        filename = "data/" + var + '_' + str(t) + 'time0_step240_oper_fc.grib'
        client = Client(source = "ecmwf", beta=True) # ecwf: last five days
        client.retrieve(
            request = {
                "date": str(t), #date start of the forecast
                "time": 0,  # time start of the forecast or 12
                "step": 240, #step of the forecast
                "stream": "oper",
                "type": "fc",
                # "levtype": "sfc",
                "param": var,
            },
            target = filename
        )
        
# # variables of probability
# for var in ['10fgg15']:  # total precipitation of at least 50mm
#     print(var)
#     for t in dates_to_download:
#         #print(var + ': ' + t)
#         filename = var + '_' + str(t) + 'time0_step240_oper_ep.grib'
#         client = Client("https://data.ecmwf.int/forecasts", beta=True) # ecwf: last five days
#         client.retrieve(
#            # date = str(t), #date start of the forecast
#             time = 0,  # time start of the forecast or 12
#             step = ["0-24"], #step of the forecast
#             stream = "enfo",
#             type = "ep",
#             param = 'tpg1'
#         )    
# print('Done!')

msl


20230823000000-240h-oper-fc.grib2:   0%|          | 0.00/282k [00:00<?, ?B/s]

20230824000000-240h-oper-fc.grib2:   0%|          | 0.00/230k [00:00<?, ?B/s]

2t


20230823000000-240h-oper-fc.grib2:   0%|          | 0.00/280k [00:00<?, ?B/s]

20230824000000-240h-oper-fc.grib2:   0%|          | 0.00/278k [00:00<?, ?B/s]

tp


20230823000000-240h-oper-fc.grib2:   0%|          | 0.00/451k [00:00<?, ?B/s]

20230824000000-240h-oper-fc.grib2:   0%|          | 0.00/450k [00:00<?, ?B/s]

# LOAD DATA

In [44]:
# LOAD AND DATA
all_variables = ['msl', '2t', 'tp']
gtp = []
gmsl = []
g2t = []
print('Loading data...')
for var in all_variables:
    print(var)
    for t in dates_to_download:
        filename = var + '_' + str(t) + 'time0_step240_oper_fc.grib'
        #plotname =  var + '_' + str(t) + 'time0_step240_oper_fc' 
        
        #output_grafic = output(output_formats = ['png'], output_name = plotname, 
        #                       output_name_first_page_number = "off" )

        #Loading Grib data
        gvar = mgrib(grib_input_file_name=filename)

        if var == 'tp':
            gtp.append(gvar)
        elif var == 'msl':
            gmsl.append(gvar)
        elif var == '2t':
            g2t.append(gvar)
print('Done!')

msl
2t
tp


In [ ]:
# Load data
def gen_raster(var):
    for t in dates_to_download:
        filename = "data/" + var + '_' + str(t) + 'time0_step240_oper_fc.grib'
        f = xr.load_dataset(filename, engine = "cfrgrib")


def load_atmdata(varlst):
    """
    varlst: str, list of str
        List containing strings of the variables to load
    """
    # gtp, gmsl, g2t = [], [], []
    # tool = [gtp, gmsl, g2t]
    for i, var in enumerate(varlst):
        print(var, " conversion")
        path = gen_raster(var)


In [5]:
filename = "data/msl_20230823time0_step240_oper_fc.grib"

In [46]:
f = xr.open_dataset(filename, engine = "cfgrib")

In [8]:
f.rio #continue from here

In [ ]:
all_variables = ['msl', '2t', 'tp'] #list of variables to load
load_atmdata(all_variables)

# WIDGETS

In [9]:
# widgets selection lat lon
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options = longitude, index = (0,len(longitude)-1), description='Longitude',
    orientation='horizontal', disabled=False)
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options = latitude, index = (0,len(latitude)-1), description='Latitude',
    orientation='horizontal', disabled=False)

# widget select sea surface variable
widget_select_ssvar = widgets.Dropdown(description='Sea surface variable', options=('msl', '2t'))

# widget select atm variable
widget_select_atmvar = widgets.Dropdown(description='Atmospheric variable', options=('tp',))
                        # la coma del final és important; sinó agafa cada lletra per separat!!
# widget select timestep
vec_timesteps = list(range(0, len(dates_to_download))) 
widget_sel_timestep = widgets.SelectionSlider(options=vec_timesteps, description='Timestep')

# FUNCTION TO PLOT

In [45]:
def update_plot(select_var, select_timestep, select_lat, select_lon):
    min_lat, max_lat = select_lat
    min_lon, max_lon = select_lon
    var = select_var
    t = select_timestep

    area = mmap(subpage_map_projection="cylindrical", subpage_lower_left_longitude=int(min_lon),
                subpage_lower_left_latitude=int(min_lat), subpage_upper_right_longitude=int(max_lon), 
                subpage_upper_right_latitude=int(max_lat)) 
 
    ##settings of the coastlines attributes
    coast = mcoast(map_coastline_land_shade = "on", map_coastline_land_shade_colour = "cream", 
                   map_grid_line_style = "dash", map_grid_colour = "brown", map_label_colour = "brown",
                   map_coastline_colour = "brown" ) 
    # CONTOUR
    contour = mcont( contour_automatic_setting = "ecmwf",legend = "on" )

    #Definition of the legend
    legend = mlegend(legend='on', legend_display_type='continuous', 
                     legend_text_colour='charcoal', legend_text_font_size=0.4)  
    title = mtext(text_lines = var, text_font_size = "0.7", text_colour = "charcoal" )

    if var == 'tp':
        aux = gtp
    elif var == 'msl':
        aux = gmsl
    elif var == '2t':
        aux = g2t
        
        
    display(plot( area, coast, aux[t], contour, title, legend))

# RESULT

In [46]:
widgets.interactive(update_plot, 
                    select_var=widget_select_ssvar, select_timestep=widget_sel_timestep,
                    select_lat=latitude_slider, 
                    select_lon=longitude_slider)

interactive(children=(Dropdown(description='Sea surface variable', options=('msl', '2t'), value='msl'), Select…

In [47]:
widgets.interactive(update_plot, 
                    select_var=widget_select_atmvar, select_timestep=widget_sel_timestep,
                    select_lat=latitude_slider, 
                    select_lon=longitude_slider)

interactive(children=(Dropdown(description='Atmospheric variable', options=('tp',), value='tp'), SelectionSlid…